In [1]:
def tidy (expr):
   converge(expr):
       distribute(expr)
       product_rule(expr)
       canonicalise(expr)
   return expr

In [2]:
def split(ex):
	split_index(ex, $\mu, m1, 4$, repeat=True)
	substitute(ex, $\partial_{4}{A??} -> 0$, repeat=True)
	substitute(ex, $\partial_{4 m?}{A??} -> 0$, repeat=True)
	substitute(ex, $\partial_{m? 4}{A??} -> 0$, repeat=True)
	canonicalise(ex)
	return ex

In [3]:
def select_index(used_indices):
	indeces = r'z w v u s r q o n m l k j i h g f e d c b a'.split() 
	for uind in indeces:
		found = False
		for qind in used_indices:
			if qind == uind:
				found = True
				break	
		if not found:
			used_indices.add(uind)
			index = uind
			break	
	return Ex(index), used_indices

In [4]:
"""Эта функция умеет брать одну ковариантную производную"""
def one_nabla(ex, used_indices): #На вход подается выражение без лишних множителей, только набла, и список уже занятых индексов
	t3, used_indices = select_index(used_indices)# здесь выбирается значок и обновляется список занятых индексов
	free = dict()# словарь тензорных значков выражения
	free['sub'] = set()
	free['up'] = set()# множества верхних и нижних свободных значков
	for nabla in ex[r'\nabla']:
		nabla.name=r'\partial' #меняем наблу на частную производную
		dindex = nabla.indices().__next__() #выбираем индекс стоящий у самой наблы
		for arg in nabla.args():             # обращаемся к аргументу производной, он один, но лучше цикл все-таки оборвать
			ret:=0;
			for index in arg.free_indices(): #собираем в словарь все свободные индексы
				if index.parent_rel==sub:
					free['sub'].add(str(index.ex()))
				else:
					free['up'].add(str(index.ex()))
			for key in free.keys(): # проходим по верхним и нижним индексам по-отдельности
				for index in free[key]:
					ind = Ex(index)
					if key == 'sub': #Строим подходящий символ Кристоффеля
						t1:= -\Gamma^{@[t3]}_{@(dindex) @[ind]};
					else:
						t1:=  \Gamma^{@[ind]}_{@(dindex) @[t3]};
					t2:= @[arg];
					for term_index in arg.free_indices(): # важное место!! тут мы проходимся по всем свободным индексам выражения и меняем одновременно все, что совпадают с индексом из словаря
						if str(term_index.ex()) == index:
							if term_index.parent_rel==sub:
								t2[term_index]:= _{@[t3]};
							else:
								t2[term_index]:= ^{@[t3]};
					ret += Ex(str(nabla.multiplier)) * t1 * t2 # строим слагаемое с Гаммой
			nabla += ret # и прибавляем его к набле, которая уже стала частной производной, важно, что nabla является узлом дерева и имеет тип ExNode, его мы можем заменить в отличии от аргумента наблы
			break
		break
	return ex, used_indices

In [5]:
"""Эта функция анализирует пришедшее к ней слагаемое из стартовой функции, а так же является рекурсией на случай, если выражение достаточно сложное и разбивается на множители, которые так же являются слагаемыми и т.д."""
def nabla_calculation(ex, used_indices, count): # на вход поступает выражение, список использованных индексов и число набл в выражении, без учета того вложенные они или перемножаются
	ex = ex.ex()
	for element in ex.top().terms(): # разбиваем выражение на слагаемые
		new:=1;
		local_count = 0
		for nabla in element[r'\nabla']: #пересчитываем наблы в каждом слагаемом
			local_count += 1
		if local_count == 0: #если их нет, то отпускаем с миром
			continue
		elif local_count == 1: # если одна, то отправляем ее раскрываться, в one_nabla можно отправлять все слагаемое, т.к. она замещает точечно узел \nabla
			new, used_indices = (one_nabla(element, used_indices))
			element.replace(new.ex())
			count -= 1
		else: #если больше одной, то проверяем произведение перед нами или нет
			if element.ex().top().name == r'\prod': # если произведение, то разбиваем его на 2 множителя (иначе произведение начинает так же раскрываться рекурсивно) и считаем наблы в них
				i = 0
				while i < 2:
					for mult in element.ex().top().children():
						local_count2 = 0
						for nabla in mult[r'\nabla']:
							local_count2 += 1
						if local_count2 == 0:
							new *= mult.ex()
						elif local_count2 == 1: # если есть 1 набла, мы ее раскрываем
							for nabla in mult[r'\nabla']:
								nabla1, used_indices = one_nabla(nabla, used_indices)
								nabla.replace(nabla1.ex())
								new *= nabla.ex()
						else:  #если больше, то отправляем на повторный анализ в рекурсию
							mult1, used_indices = nabla_calculation(mult, used_indices, local_count)
							new *= mult1
						i+=1
				new *= Ex(str(element.multiplier))
				element.replace(new)				
			else: # если не произведение, то берем аргумент и отправляем его в рекурсию, выражение перестраиваем, т.к. заменять можно только узлы
				for nabla in element[r'\nabla']:
					for arg1 in nabla.args():
						arg2, used_indices = nabla_calculation(arg1, used_indices, count - 1)
						index = nabla.indices().__next__() 
						t := \nabla_{@(index)}{@[arg2]};
					new = t
					nabla1, used_indices = one_nabla(new, used_indices)
					new = Ex(str(nabla.multiplier)) * nabla1
					nabla.replace(new)
	return ex, used_indices

In [6]:
def expand_nabla(ex): #Основная входная функция, проверяет выражение на то, является ли оно равенством и разбивает его на слагаемые, по пути считает наблы
	if ex.top().name == '\equals':
		for child in ex.top().children():
			ret:=0;
			for element in child.ex().top().terms():
				count = 0
				used_indices = set()
				for nabla in element.ex()[r'\nabla']:
					count += 1
				if count == 0:
					ret += element.ex()
		#new_ex += element.ex()
				else:
					for n in element.ex():
						for index in n.indices():
							used_indices.add(str(index))
		#for nabla in element[r'\nabla']:
					element1, used_indices = nabla_calculation(element, used_indices, count)
					ret +=element1
			child.replace(ret)
	else:
		for element in ex.top().terms():
				count = 0
				used_indices = set()
				for nabla in element.ex()[r'\nabla']:
					count += 1
				if count == 0:
					pass
				else:
					for n in element.ex():
						for index in n.indices():
							used_indices.add(str(index))
				element1, used_indices = nabla_calculation(element, used_indices, count)
				element.replace(element1)
	return ex

In [7]:
def kaluza(ex):
	substitute(ex, $g_{4 4} -> \phi**{2}$ )
	substitute(ex, $g_{m 4} -> \phi**{2} A_{m}$ )
	substitute(ex, $g_{4 m} -> \phi**{2} A_{m}$ )
	substitute(ex, $g_{m n} -> h_{m n} + \phi**{2} A_{m} A_{n}$ )
	substitute(ex, $g^{4 4} -> \phi**{-2} +  A_{m} h^{m n} A_{n}$ )
	substitute(ex, $g^{m 4} -> - h^{m n} A_{n}$ )
	substitute(ex, $g^{4 m} -> - h^{m n} A_{n}$ )
	substitute(ex, $g^{m n} -> h^{m n}$ )
	return ex

In [8]:
def writing(ex):
	with open("output", "a") as file:
   		file.write("\n" + ex.input_form() + "\n")